# More Value Expressions
Let's walk through some more value expressions.

## Setup

In [ ]:
!curl -LsS -o geography.db 'https://storage.googleapis.com/ibis-tutorial-data/geography.db'

In [ ]:
import os
import tempfile

import ibis

ibis.options.interactive = True

connection = ibis.sqlite.connect(
    'geography.db'
)

## Type casting

The <a href='../../api/datatypes'>Ibis type system</a> supports the most common data types used in analytics, including support for nested types like lists, structs, and maps.

Type names can be used to cast from one type to another.

In [ ]:
countries = connection.table('countries')
countries

In [ ]:
countries = connection.table('countries')
countries.population.cast('float').sum()

In [ ]:
countries.area_km2.cast('int32').sum()

## Case / if-then-else expressions


We support a number of variants of the SQL-equivalent `CASE` expression, and will add more API functions over time to meet different use cases and enhance the expressiveness of any branching-based value logic.

In [ ]:
expr = (
    countries.continent.case()
    .when('AF', 'Africa')
    .when('AN', 'Antarctica')
    .when('AS', 'Asia')
    .when('EU', 'Europe')
    .when('NA', 'North America')
    .when('OC', 'Oceania')
    .when('SA', 'South America')
    .else_(countries.continent)
    .end()
    .name('continent_name')
)

expr.value_counts()

If the `else_` default condition is not provided, any values not matching one of the conditions will be `NULL`.

In [ ]:
expr = (
    countries.continent.case()
    .when('AF', 'Africa')
    .when('AS', 'Asia')
    .when('EU', 'Europe')
    .when('NA', 'North America')
    .when('OC', 'Oceania')
    .when('SA', 'South America')
    .end()
    .name('continent_name_with_nulls')
)

expr.value_counts()

To test for an arbitrary series of boolean conditions, use the `case` API method and pass any boolean expressions potentially involving columns of the table:

In [ ]:
expr = (
    ibis.case()
    .when(countries.population > 25_000_000, 'big')
    .when(countries.population < 5_000_000, 'small')
    .else_('medium')
    .end()
    .name('size')
)

countries['name', 'population', expr].limit(10)

Simple ternary-cases (like the Python `X if COND else Y`) can be written using the `ifelse` function:

In [ ]:
expr = (countries.continent == 'AS').ifelse('Asia', 'Not Asia').name('is_asia')

countries['name', 'continent', expr].limit(10)

## Set membership


The `isin` and `notin` functions are like their pandas counterparts. These can take:

- A list of value expressions, either literal values or other column expressions
- An array/column expression of some kind

In [ ]:
is_america = countries.continent.isin(['NA', 'SA'])
countries[is_america].continent.value_counts()

You can also check for membership in an array. Here is an example of filtering based on the top 3 (ignoring ties) most frequently-occurring values in the `string_col` column of alltypes:

In [ ]:
top_continents = countries.continent.value_counts().limit(3).continent
top_continents_filter = countries.continent.isin(top_continents)
expr = countries[top_continents_filter]

expr.count()

This is a common enough operation that we provide a special analytical filter function `topk`:

In [ ]:
countries.continent.topk(3)

Cool, huh? More on `topk` later.

## Null Checking

Like their pandas equivalents, the `isnull` and `notnull` functions return True values if the values are null, or non-null, respectively. For example:

In [ ]:
expr = (
    countries.continent.case()
    .when('AF', 'Africa')
    .when('EU', 'Europe')
    .when('AS', 'Asia')
    .end()
    .name('top_continent_name')
)

expr.isnull().value_counts()

Functions like `isnull` can be combined with `case` expressions or functions like `ifelse` to replace null values with some other value. `ifelse` here will use the first value supplied for any `True` value and the second value for any `False` value. Either value can be a scalar or array. 

In [ ]:
expr2 = expr.isnull().ifelse('Other continent', expr).name('continent')
expr2.value_counts()

## Distinct-based operations


Ibis supports using `distinct` to remove duplicate rows or values on tables or arrays. For example:

In [ ]:
countries[['continent']].distinct()

This can be combined with `count` to form a reduction metric:

In [ ]:
metric = countries[['continent']].distinct().count().name('num_continents')
metric

## String operations


What's supported is pretty basic right now. We intend to support the full gamut of regular expression munging with a nice API, though in some cases some work will be required on SQLite's backend to support everything. 

In [ ]:
countries[['name']].limit(5)

At the moment, basic substring operations (`substr`, with conveniences `left` and `right`) and Python-like APIs such as `lower` and `upper` (for case normalization) are supported. So you could count first letter occurrences in a string column like so:

In [ ]:
expr = countries.name.lower().left(1).name('first_letter')
expr.value_counts().order_by(('count', False)).limit(10)

For fuzzy and regex filtering/searching, you can use one of the following

- `like`, works as the SQL `LIKE` keyword
- `rlike`, like `re.search` or SQL `RLIKE`
- `contains`, like `x in str_value` in Python

In [ ]:
countries[countries.name.like('%GE%')].name

In [ ]:
countries[countries.name.lower().rlike('.*ge.*')].name

In [ ]:
countries[countries.name.lower().contains('ge')].name

## Timestamp operations


Date and time functionality is relatively limited at present compared with pandas, but we'll get there. The main things we have right now are

- Field access (year, month, day, ...)
- Timedeltas
- Comparisons with fixed timestamps

In [ ]:
independence = connection.table('independence')

independence[
    independence.independence_date,
    independence.independence_date.month().name('month'),
].limit(10)

Somewhat more comprehensively

In [ ]:
def get_field(f):
    return getattr(independence.independence_date, f)().name(f)


fields = [
    'year',
    'month',
    'day',
]  # datetime fields can also use: 'hour', 'minute', 'second', 'millisecond'
projection = [independence.independence_date] + [get_field(x) for x in fields]
independence[projection].limit(10)

For timestamp arithmetic and comparisons, check out functions in the top level `ibis` namespace. This include things like `day` and `second`, but also the `ibis.timestamp` function:

In [ ]:
independence[
    independence.independence_date.min(),
    independence.independence_date.max(),
    independence.count().name('nrows'),
].distinct()

In [ ]:
independence[independence.independence_date > '2000-01-01'].count()

Some backends support adding offsets. For example:

```python
independence.independence_date + ibis.interval(days=1)
ibis.now() - independence.independence_date
```